In [1]:
import numpy as np
import netCDF4 as nc
import matplotlib.pyplot as plt
import os
from os import listdir
from os.path import isfile, join
import glob

In [3]:
# DEFINITIONS
arch_dir= '/archive/uda/CMIP6/'

# Simulation locations and storage
tim_sims = {'ssp585':{'nm':'ssp585','dir':'ScenarioMIP/NOAA-GFDL/GFDL-ESM4/ssp585','yr_inc':2},
            'ssp585':{'nm':'ssp585','dir':'ScenarioMIP/NOAA-GFDL/GFDL-ESM4/ssp585','yr_inc':2
            'his':   {'nm':'historical','dir':'CMIP/NOAA-GFDL/GFDL-ESM4/historical','yr_inc':5},
            'piC':   {'nm':'piControl','dir':'CMIP/NOAA-GFDL/GFDL-ESM4/piControl','yr_inc':5}
            }    

#Omon_GFDL-ESM4_historical_r1i1p1f1_gr

var_nms = {'co3satarag'}
    
ts_yrs = {'ssp585':{'yr_1':2061,'yr_n':2100},
          'his':{'yr_1':1975,'yr_n':2014}, 
          'piC':   {'yr_1':101,'yr_n':351}}


In [6]:
# DMGET Other Nutrient limitations
for tim_sim in tim_sims:

    for var in var_nms:
        sub_str = '/r1i1p1f1/Omon/' + var + '/gr/'
        
        # Select latest version downloaded for given variable 
        ver_str = os.listdir((arch_dir + tim_sims[tim_sim]['dir'] + sub_str))[0] + '/'
        
        ts_str = var + '_Omon_GFDL-ESM4_' + tim_sims[tim_sim]['nm'] + '_r1i1p1f1_gr_'
        fil_dir = arch_dir + tim_sims[tim_sim]['dir'] + sub_str + ver_str + ts_str
        
        fils = fil_dir + '*.nc'
        
        print(fils)
        # NOTE: with '&', difficult to tell when available for ncks. 
        #       Check for hangups by running second dmget w/o '&'
        
        dmget_fnc_str = 'dmget ' + fils + ' &' 
        
        echo_fnc_str = 'echo dmget ' + tim_sim + ' ' + var

        #OS COMMAND
        os.system(echo_fnc_str)
        os.system(dmget_fnc_str)

/archive/uda/CMIP6/ScenarioMIP/NOAA-GFDL/GFDL-ESM4/ssp585/r1i1p1f1/Omon/co3satarag/gr/v20180701/co3satarag_Omon_GFDL-ESM4_ssp585_r1i1p1f1_gr_*.nc
dmget ssp585 co3satarag
/archive/uda/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/historical/r1i1p1f1/Omon/co3satarag/gr/v20190726/co3satarag_Omon_GFDL-ESM4_historical_r1i1p1f1_gr_*.nc
dmget his co3satarag
/archive/uda/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/piControl/r1i1p1f1/Omon/co3satarag/gr/v20180701/co3satarag_Omon_GFDL-ESM4_piControl_r1i1p1f1_gr_*.nc
dmget piC co3satarag


In [7]:
# NCO PROCESS FILES for TIME SERIES
for tim_sim in tim_sims:
    yr_inc = tim_sims[tim_sim]['yr_inc']
        
    # time loop start yr, end yr
    yr0 = ts_yrs[tim_sim]['yr_1']
    yrn = ts_yrs[tim_sim]['yr_n']
    
    for var in var_nms:
        sub_str = '/r1i1p1f1/Omon/' + var + '/gr/'
        
        # Select latest version downloaded for given variable 
        ver_str = os.listdir((arch_dir + tim_sims[tim_sim]['dir'] + sub_str))[0] + '/'
        
        ts_str = var + '_Omon_GFDL-ESM4_' + tim_sims[tim_sim]['nm'] + '_r1i1p1f1_gr_'
        fil_dir = arch_dir + tim_sims[tim_sim]['dir'] + sub_str + ver_str + ts_str
        
        src_fils = fil_dir + '*.nc'
        var_files = glob.glob(src_fils)
        
        print(src_fils)
        #iterate over the files
        for fil_txt in var_files: 
            fyr0 = int(fil_txt.split('01-')[0][-4:])
            fyrn = int(fil_txt.split('12.')[0][-4:])
    
            # include all data if file is between yr0 and yrn of climatology
            if ((yr0<=fyr0) & (fyrn<=yrn)):
                # ncks all times
                #print(fil_txt)
                hyp_str = 'ncks -O '
                
            # include overlapping years at the beginning of the climatology
            elif ((fyr0<yr0) & (yr0<=fyrn) & (fyrn<=yrn)):
                tidx = 12*(yr0-fyr0)
                tidxn = tidx + 12*(fyrn-yr0+1) -1
                tim_str = ('-d time,' + str(tidx) + ',' + str(tidxn) + ' ')
                hyp_str = 'ncks -O ' + tim_str 
            
            elif ((fyr0<yr0) & (yr0<=fyrn) & (yrn<=fyrn)):
                tidx = 12*(yr0-fyr0)
                tidxn = tidx + 12*(yrn-yr0+1) -1
                tim_str = ('-d time,' + str(tidx) + ',' + str(tidxn) + ' ')
                hyp_str = 'ncks -O ' + tim_str 
                    
            # include overlapping years at the end of the climatology (often only for piC)   
            elif ((fyr0<=yrn) & (yrn<fyrn)):
                tidx = 12*(yrn-fyr0+1)-1
                tim_str = ('-d time,,' + str(tidx) + ' ')
                hyp_str = 'ncks -O ' + tim_str 
                
            # skip files if outside of the clim range
            else:
                continue
                
            # temporary destination file naming - which ever represents the first year in the file
            dfy = np.max((fyr0,yr0))
                
            dest_fil_str1 =  var + '_'
            dest_fil = dest_fil_str1 + str(dfy).zfill(4) + '_' + tim_sim +'_tmp.nc'
            nco_fnc_str = hyp_str + '-v ' + var + ',lat_bnds,lon_bnds ' + fil_txt + ' ' + dest_fil
                
            print(nco_fnc_str)
            # the nco command
            os.system(nco_fnc_str)
            #print(dest_fil)
            
        dest_fil_all = dest_fil_str1 + str(yr0).zfill(4) + '-' + str(yrn).zfill(4) + '_' + tim_sim + '_ts.nc'
        nrcat_fnc_str = 'ncrcat -O ' + dest_fil_str1 + '????_' + tim_sim + '_tmp.nc ' + dest_fil_all
            
        os.system(nrcat_fnc_str)
            
        print()
        
        rm_fnc_str = 'rm ' + dest_fil_str1 + '????_' + tim_sim + '_tmp.nc'
        # OS COMMAND
        os.system(rm_fnc_str)

/archive/uda/CMIP6/ScenarioMIP/NOAA-GFDL/GFDL-ESM4/ssp585/r1i1p1f1/Omon/co3satarag/gr/v20180701/co3satarag_Omon_GFDL-ESM4_ssp585_r1i1p1f1_gr_*.nc
ncks -O -v co3satarag,lat_bnds,lon_bnds /archive/uda/CMIP6/ScenarioMIP/NOAA-GFDL/GFDL-ESM4/ssp585/r1i1p1f1/Omon/co3satarag/gr/v20180701/co3satarag_Omon_GFDL-ESM4_ssp585_r1i1p1f1_gr_209501-210012.nc co3satarag_2095_ssp585_tmp.nc
ncks -O -v co3satarag,lat_bnds,lon_bnds /archive/uda/CMIP6/ScenarioMIP/NOAA-GFDL/GFDL-ESM4/ssp585/r1i1p1f1/Omon/co3satarag/gr/v20180701/co3satarag_Omon_GFDL-ESM4_ssp585_r1i1p1f1_gr_207501-209412.nc co3satarag_2075_ssp585_tmp.nc
ncks -O -d time,72,239 -v co3satarag,lat_bnds,lon_bnds /archive/uda/CMIP6/ScenarioMIP/NOAA-GFDL/GFDL-ESM4/ssp585/r1i1p1f1/Omon/co3satarag/gr/v20180701/co3satarag_Omon_GFDL-ESM4_ssp585_r1i1p1f1_gr_205501-207412.nc co3satarag_2061_ssp585_tmp.nc

/archive/uda/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/historical/r1i1p1f1/Omon/co3satarag/gr/v20190726/co3satarag_Omon_GFDL-ESM4_historical_r1i1p1f1_gr_*.nc
ncks -O